**Import bibliotek + instalacja**

In [49]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [50]:
!pip install langchain sentence-transformers faiss-cpu pypdf transformers torch langchain-community PyPDF2 #instalujemy potrzebne biblioteki

In [51]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM, set_seed
from langchain.chains import RetrievalQA
from PyPDF2 import PdfReader

#Ustawiam seed do powtarzalności wyników
set_seed(42)

**Podział tekstu**

Zdecydowałem się na podanie jedynie 3 artykułów (temat: półprzewodniki), spodziewam się że w większości nie będzie mi w stanie odpowiedzieć na wszystkie moje pytania, aczkolwiek jestem ciekaw na co będzie w stanie odpowiedzieć a na co nie.   

In [52]:
# Lista PDF-ów
pdf_files = [
    "/content/drive/MyDrive/Semiconductors_History_and_Applications.pdf",
    "/content/drive/MyDrive/Defect_modeling_in_semiconductors_the_role_of_firs.pdf",
    "/content/drive/MyDrive/Optimization_of_Thermal_Stress_in_High-Power_Semic.pdf"
]

all_docs = []

# Pętla przetwarzania PDF-ów
for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        # Podział tekstu
        chunk_size=500,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    docs = text_splitter.split_documents(pages)
    all_docs.extend(docs)

print(f"Załadowano {len(all_docs)} fragmentów z {len(pdf_files)} artykułów")


Załadowano 695 fragmentów z 3 artykułów


Dobrany model embeddingowy: "sentence-transformers/multi-qa-mpnet-base-dot-v1"
Czytając z dokumentacji służy do wyszukiwania semantycznego, co z kolei powinno być dobrym wyborem modelu do tzw. asystenta Q&A.

In [53]:
# Nowy model embeddingów
model_name_embed = "sentence-transformers/multi-qa-mpnet-base-dot-v1"  #"sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name_embed)

# FAISS
db = FAISS.from_documents(all_docs, embeddings)

# Zapis lokalny
db.save_local("faiss_index_mpnet")

**Sprawdzenie semantyczne**

Sprawdzam jak model zareaguję na dane zapytanie, w postaci fragmentów wziętych z artykułów.

In [54]:
query = "What is a semiconductor?"
# Top 5 z 20 wybranych (Podobne informacje)
similar_docs = db.similarity_search(query, k=5, fetch_k = 20)
for doc in similar_docs:
    print(doc.page_content[:500] + "...\n---")

Semiconductors: History and Applications
DOI: http://dx.doi.org/10.5772/intechopen.1008474
17
References
[1] Fu Y , Fu Y . Semiconductor materials. 
In: Physical Models of Semiconductor 
Quantum Devices. Dordrecht, 
Netherlands: Springer; 2014. pp. 1-66. 
DOI: 10.1007/978-94-007-7174-1
[2] Busch G. Early history of the physics 
and chemistry of semiconductors-
from doubts to fact in a hundred years. 
European Journal of Physics. 1989;10:254
[3] Pearson GL, Brattain WH. History of...
---
15
Semiconductors: History and Applications
DOI: http://dx.doi.org/10.5772/intechopen.1008474
has led to a transformative shift in product design, establishing a substantial market 
for semiconductors at the beginning of the twenty-first century [10].
Today, the outstanding characteristics of semiconductor nanomaterials have 
sparked significant anticipation and excitement in a broad variety of sophisticated 
technologies, including nanoelectronics, imaging devices, sensors, nonlinear and...
---
Semicon

Model nie był w stanie podać mi poprawnej definicji półprzewodnika, zamiast tego znajdywał fragmenty najprawdopodobniej tytułowe w których były podobne wpisy, nie jest to wynik zadowolujący, więc spróbuję inne podejście. Będę przeszukiwać znowuż to samo pytanie ale wymuszę żeby podawał nowe informację na jego temat.

In [55]:
# Top 5 z 20 wybranych (Różne informacje)
diffr_docs = db.max_marginal_relevance_search(query, k=5, fetch_k=20)
for doc in diffr_docs:
    print(doc.page_content[:500] + "...\n---")

Semiconductors: History and Applications
DOI: http://dx.doi.org/10.5772/intechopen.1008474
17
References
[1] Fu Y , Fu Y . Semiconductor materials. 
In: Physical Models of Semiconductor 
Quantum Devices. Dordrecht, 
Netherlands: Springer; 2014. pp. 1-66. 
DOI: 10.1007/978-94-007-7174-1
[2] Busch G. Early history of the physics 
and chemistry of semiconductors-
from doubts to fact in a hundred years. 
European Journal of Physics. 1989;10:254
[3] Pearson GL, Brattain WH. History of...
---
1.1 Definition, characteristics
The term semiconductor is composed of two words: semi and conductor. Semi 
means imperfect while conductive refers to something that can conduct electricity. 
In this regard, semiconductors are materials with properties between conductors and 
insulators that have the capability of conducting electric current.
The temperature dependency of semiconductor conductivity is one of their most...
---
materials used in this process. The resulting products are printed circuit boar



*   Widoczne jest, że pojawiła się definicja półprzewodnika (w sekcji: 1.1 Definition [...]) a zatem model jest w stanie zlokalizować gdzie może być odpowiedź.  
*   Dodatkowo widać że model nie radzi sobie ze wzorami matematycznymi (a zatem najprawdopodobniej nie radzi sobie z LaTeX-em), co raczej wymagałoby zastosowanie dodatkowej bilioteki.



Zatem pozostanę na retriver-rze jako "MMR".

**Utworzenie modelu**

Model przeze mnie dobrany do sprawdzenia: "google/flan-t5-base". Czytając dokumentację w sekcji "Uses" również był określony jako model do odpowiadania na pytania.

In [71]:
model_name_qa = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_qa)
tokenizer = AutoTokenizer.from_pretrained(model_name_qa)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100, # Niekiedy wiadomości modelu były za długie, dlatego ustawiam limit
    truncation=True,
    device=-1
)

llm = HuggingFacePipeline(pipeline=pipe)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)
result = qa_chain.run("Define a semiconductor")
print(f"AI Assistant: {result}")


Device set to use cpu


AI Assistant: The term semiconductor is composed of two words: semi and conductor. Semi means imperfect while conductive refers to something that can conduct electricity. In this regard, semiconductors are materials with properties between conductors and insulators that have the capability of conducting electric current. The temperature dependency of semiconductor conductivity is one of their most Semiconductors: History and Applications DOI: http://dx.doi.org/10.5772/intechopen.1008474



**Cykl Q&A**

In [72]:
print("Ready! (type: 'q' to exit!)\n")

while True:
    query = input("You: ")
    if query.lower() in ["q"]:
        print("End Q&A")
        break

    result = qa_chain.invoke(query)
    print(f"AI: {result['result']}\n")


Ready! (type: 'q' to exit!)

You: Define semiconductor
AI: The term semiconductor is composed of two words: semi and conductor. Semi means imperfect while conductive refers to something that can conduct electricity. In this regard, semiconductors are materials with properties between conductors and insulators that have the capability of conducting electric current. The temperature dependency of semiconductor conductivity is one of their most Semiconductors: History and Applications DOI: http://dx.doi.org/10.5772/intechopen.1008474

You: What is a semiconductor
AI: Semiconductors are materials with properties between conductors and insulators that have the capability of conducting electric current.

You: What is semiconductor?
AI: Wu Y , Li Y . Semiconductors. In: Physical Models of Semiconductor Quantum Devices. Dordrecht, Netherlands: Springer; 2014. pp. 1-66. DOI: 10.1007/978-94-007-7174-1 [2] Busch G. Early history of the physics and chemistry of semiconductors- from doubts to fact 

Przechodząc po kolei po pytaniach zadawanych przeze mnie:

 "moje_pytanie" -> wnioski z wygenerowanej odpowidzi

*   "Define semiconductor" -> Generalny opis półprzewodnika, z dodatkowymi informacjami, spodziewam się że ponieważ model był wytrenowany żeby odpowiadać na polecenia "Explain", "Define" itp. i dlatego też próbuję przedstawić dogłębny opis pojęcia (gdzie i tak ostatecznie nie był w stanie tego zrobić, najprawdopodobniej przez to jak został podzielony tekst dla niego).
*   "What is semiconductor" -> Poprawna odpowiedź!
*   "What is semiconductor?" -> Zaskakująca odpowiedź najprawdopodobniej "?" zupełnie zmienił embedding przez co model nie odpowiedział zgodnie z oczekiwaniem, dodatkowo wydaję mi się że dana odpowiedź wygenerowała się przez to że podobny tytuł publikacji mógł tam wystąpić.
*   "What is conductor" -> Poprawna odpowiedź!
*   "Define conductor" -> Przez dogłębną analizę myli pojęcia "conductor" i "semiconductor", również ze względu na brak danych dla modelu.
*   "What defects can be present in semiconductors" -> Poprawna odpowiedź!
*   "What defects are in materials overall" -> Nie dostał danych na ten temat a więc wymienia co ma.
*   "Can you say something nice" -> Losowa odpowiedź.
*   "What is 9 + 10" -> Nie wie jak liczyć :(












**Wnioski**

Budując jakiegokolwiek "asystenta AI" liczy się z bardzo dużą ilością parametrów, w głównej mierze wydaję mi się że dane władowane do przeanalizowania ich podział oraz embeddingi mają jeden z największych wpływów co by również potwierdzało niekiedy dziwne odpowiedzi oraz banalną niewiedzę mojego "asystenta".

Poza tym to zauważyłem że każda odpowiedź którą generował "asystent" najczęściej brała sam tekst z artykułów, natomiast spodziewam się że jest to spowodowane konkretnym doborem modelu.

Dodatkowo wydaję mi się ważnym parametrem czas pracy który szedł na iterację testowania, w przypadku użycia "GPU" nie ma z tym w głównej mierze problemu (większość kodu robiła się w kilka sekund), co z kolei nie można powiedzieć na temat "CPU" (większośc operacji trwały niejakie minuty a nawet dłużej!).